# Interaction Event Handling Demo

This notebook demonstrates how to use the `on_interaction()` callback to respond to user interactions with the Cesium viewer.

The callback captures:
- **Camera movements** (pan, zoom, rotate)
- **Click events** (left and right clicks)
- **Timeline scrubbing** (when animation timeline is enabled)

Each interaction provides detailed state information including camera position, timestamps, and clicked positions.

In [ ]:
from cesiumjs_anywidget import CesiumWidget
from datetime import datetime
import json

## 1. Basic Interaction Tracking

Let's start with a simple example that prints all interaction events.

In [ ]:
# Create a widget centered on Paris
widget1 = CesiumWidget(
    latitude=48.8566,
    longitude=2.3522,
    altitude=5000,
    height="500px"
)

# Define a simple callback that prints all events
def log_all_interactions(event):
    print(f"\n{'='*60}")
    print(f"Interaction Type: {event['type']}")
    print(f"Timestamp: {event['timestamp']}")
    print(f"Camera: lat={event['camera']['latitude']:.4f}, lon={event['camera']['longitude']:.4f}, alt={event['camera']['altitude']:.0f}m")
    
    if 'picked_position' in event:
        pos = event['picked_position']
        print(f"Clicked position: lat={pos['latitude']:.4f}, lon={pos['longitude']:.4f}, alt={pos['altitude']:.0f}m")
    
    if 'picked_entity' in event:
        print(f"Clicked entity: {event['picked_entity']}")

# Register the callback
widget1.on_interaction(log_all_interactions)

widget1

**Try this:**
- Pan the camera around
- Zoom in/out
- Click on different locations
- Right-click on the map

You'll see detailed information about each interaction printed below.

## 2. Camera Position Tracking

Track and store camera positions as the user explores the scene.

In [ ]:
# Create a widget
widget2 = CesiumWidget(
    latitude=40.7128,
    longitude=-74.0060,
    altitude=10000,
    height="500px"
)

# Store camera positions
camera_history = []

def track_camera_movements(event):
    if event['type'] == 'camera_move':
        camera_history.append({
            'timestamp': event['timestamp'],
            'latitude': event['camera']['latitude'],
            'longitude': event['camera']['longitude'],
            'altitude': event['camera']['altitude'],
            'heading': event['camera']['heading'],
            'pitch': event['camera']['pitch'],
            'roll': event['camera']['roll']
        })
        print(f"Recorded position #{len(camera_history)}: ({event['camera']['latitude']:.4f}, {event['camera']['longitude']:.4f})")

widget2.on_interaction(track_camera_movements)

widget2

**Move the camera around, then run the cell below to see the history:**

In [ ]:
print(f"\nRecorded {len(camera_history)} camera positions:")
print("\nFirst 5 positions:")
for i, pos in enumerate(camera_history[:5]):
    print(f"{i+1}. ({pos['latitude']:.4f}, {pos['longitude']:.4f}) at altitude {pos['altitude']:.0f}m")

if len(camera_history) > 5:
    print(f"\n... and {len(camera_history) - 5} more")

## 3. Click Event Handler

Capture and visualize clicked locations on the map.

In [ ]:
# Create widget
widget3 = CesiumWidget(
    latitude=51.5074,
    longitude=-0.1278,
    altitude=5000,
    height="500px"
)

# Store clicked points
clicked_points = []

def handle_clicks(event):
    if event['type'] in ['left_click', 'right_click']:
        if 'picked_position' in event:
            pos = event['picked_position']
            clicked_points.append({
                'type': event['type'],
                'latitude': pos['latitude'],
                'longitude': pos['longitude'],
                'altitude': pos['altitude'],
                'timestamp': event['timestamp']
            })
            
            click_type = "Left" if event['type'] == 'left_click' else "Right"
            print(f"✓ {click_type} click #{len(clicked_points)}: ({pos['latitude']:.4f}, {pos['longitude']:.4f}, {pos['altitude']:.0f}m)")

widget3.on_interaction(handle_clicks)

widget3

**Click on different locations, then visualize them as GeoJSON:**

In [ ]:
# Convert clicked points to GeoJSON and display them
if clicked_points:
    geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [point['longitude'], point['latitude'], point['altitude']]
                },
                "properties": {
                    "name": f"Click {i+1}",
                    "click_type": point['type'],
                    "timestamp": point['timestamp']
                }
            }
            for i, point in enumerate(clicked_points)
        ]
    }
    
    widget3.load_geojson(geojson)
    print(f"✓ Displayed {len(clicked_points)} clicked points on the map")
else:
    print("No points clicked yet. Click on the map above!")

## 4. Filtering by Interaction Type

Process only specific types of interactions.

In [ ]:
widget4 = CesiumWidget(
    latitude=35.6762,
    longitude=139.6503,
    altitude=8000,
    height="500px"
)

# Separate counters for each interaction type
interaction_counts = {
    'camera_move': 0,
    'left_click': 0,
    'right_click': 0,
    'timeline_scrub': 0
}

def count_interactions(event):
    event_type = event['type']
    interaction_counts[event_type] = interaction_counts.get(event_type, 0) + 1
    
    if event_type == 'camera_move':
        print(f"📹 Camera moved (#{interaction_counts['camera_move']})")
    elif event_type == 'left_click':
        print(f"🖱️  Left click (#{interaction_counts['left_click']})")
    elif event_type == 'right_click':
        print(f"🖱️  Right click (#{interaction_counts['right_click']})")
    elif event_type == 'timeline_scrub':
        print(f"⏱️  Timeline scrubbed (#{interaction_counts['timeline_scrub']})")

widget4.on_interaction(count_interactions)

widget4

In [ ]:
# View interaction statistics
print("\nInteraction Statistics:")
print("="*40)
for interaction_type, count in interaction_counts.items():
    print(f"{interaction_type:20s}: {count:4d}")
print("="*40)
print(f"{'Total':20s}: {sum(interaction_counts.values()):4d}")

## 5. Timeline Interaction Demo

When using CZML data with time-dynamic features, you can track timeline scrubbing events.

In [ ]:
# Create a simple CZML document with time-dynamic content
czml = [
    {
        "id": "document",
        "version": "1.0",
        "clock": {
            "interval": "2024-01-01T00:00:00Z/2024-01-01T12:00:00Z",
            "currentTime": "2024-01-01T00:00:00Z",
            "multiplier": 60,
            "range": "LOOP_STOP"
        }
    },
    {
        "id": "moving_point",
        "name": "Moving Point",
        "availability": "2024-01-01T00:00:00Z/2024-01-01T12:00:00Z",
        "position": {
            "epoch": "2024-01-01T00:00:00Z",
            "cartographicDegrees": [
                0, 48.8566, 2.3522, 1000,
                21600, 48.8566, 2.3722, 1000,
                43200, 48.8766, 2.3522, 1000
            ]
        },
        "point": {
            "pixelSize": 10,
            "color": {"rgba": [255, 0, 0, 255]}
        }
    }
]

widget5 = CesiumWidget(
    latitude=48.8566,
    longitude=2.3522,
    altitude=10000,
    height="500px",
    show_timeline=True
)

widget5.load_czml(czml)

# Track timeline interactions
timeline_events = []

def track_timeline(event):
    if event['type'] == 'timeline_scrub' and 'clock' in event:
        clock = event['clock']
        timeline_events.append({
            'current_time': clock['current_time'],
            'multiplier': clock['multiplier'],
            'is_animating': clock['is_animating']
        })
        print(f"⏱️  Timeline at {clock['current_time']}, speed={clock['multiplier']}x, animating={clock['is_animating']}")
    
    # Also log other interactions
    if event['type'] in ['left_click', 'right_click']:
        print(f"🖱️  {event['type']} detected")

widget5.on_interaction(track_timeline)

widget5

**Try scrubbing the timeline or clicking play/pause to see timeline events!**

In [ ]:
# View timeline event history
if timeline_events:
    print(f"\nRecorded {len(timeline_events)} timeline events")
    print("\nFirst 3 events:")
    for i, event in enumerate(timeline_events[:3]):
        print(f"{i+1}. Time: {event['current_time']}, Speed: {event['multiplier']}x")
else:
    print("No timeline events yet. Try scrubbing the timeline!")

## 5.5. Clock Timestamp Tracking

Every interaction event includes **two different timestamps**:
- `timestamp`: System time when the interaction occurred
- `clock.current_time`: Cesium clock's simulation time

This is especially useful for time-dynamic visualizations where you need to correlate user interactions with the simulation timeline.

In [ ]:
# Create a widget with time-dynamic CZML data
# Define a realistic satellite orbit around Earth
import math

# Generate satellite orbit positions (circular orbit with 16 points)
def generate_orbit_positions(num_points=16, altitude=400000, inclination_deg=30):
    """Generate positions for a circular satellite orbit."""
    positions = []
    duration = 5400  # 90 minutes orbit in seconds (typical LEO orbit period)
    
    for i in range(num_points + 1):  # +1 to complete the circle
        t = (duration * i) / num_points
        angle = (360.0 * i) / num_points
        
        # Calculate position with inclination
        lon = angle % 360
        lat = inclination_deg * math.sin(math.radians(angle))
        
        positions.extend([t, lon, lat, altitude])
    
    return positions

orbit_positions = generate_orbit_positions()

czml_clock = [
    {
        "id": "document",
        "version": "1.0",
        "clock": {
            "interval": "2024-06-01T00:00:00Z/2024-06-01T01:30:00Z",
            "currentTime": "2024-06-01T00:00:00Z",
            "multiplier": 30,  # 30x speed
            "range": "LOOP_STOP"
        }
    },
    {
        "id": "satellite",
        "name": "Satellite",
        "availability": "2024-06-01T00:00:00Z/2024-06-01T01:30:00Z",
        "position": {
            "epoch": "2024-06-01T00:00:00Z",
            "cartographicDegrees": orbit_positions
        },
        "point": {
            "pixelSize": 15,
            "color": {"rgba": [0, 255, 0, 255]}
        },
        "label": {
            "text": "Satellite",
            "font": "14pt sans-serif",
            "fillColor": {"rgba": [255, 255, 255, 255]},
            "outlineColor": {"rgba": [0, 0, 0, 255]},
            "outlineWidth": 2,
            "pixelOffset": {"cartesian2": [0, 20]}
        },
        "path": {
            "material": {
                "solidColor": {
                    "color": {"rgba": [0, 255, 0, 128]}
                }
            },
            "width": 2,
            "leadTime": 0,
            "trailTime": 5400,
            "resolution": 5
        }
    }
]

widget_clock = CesiumWidget(
    latitude=0,
    longitude=90,
    altitude=2000000,
    height="500px",
    show_timeline=True
)

widget_clock.load_czml(czml_clock)

# Track both timestamps
clock_events = []

def compare_timestamps(event):
    system_time = event['timestamp']
    cesium_time = event['clock']['current_time'] if event.get('clock') else 'N/A'
    
    clock_events.append({
        'type': event['type'],
        'system_time': system_time,
        'cesium_time': cesium_time,
        'clock_multiplier': event['clock']['multiplier'] if event.get('clock') else None,
        'is_animating': event['clock']['is_animating'] if event.get('clock') else None
    })
    
    print(f"\n{event['type'].upper()}")
    print(f"  System time:  {system_time}")
    print(f"  Cesium time:  {cesium_time}")
    if event.get('clock'):
        print(f"  Speed:        {event['clock']['multiplier']}x")
        print(f"  Animating:    {event['clock']['is_animating']}")

widget_clock.on_interaction(compare_timestamps)

print("Interact with the viewer and watch the satellite orbit Earth!")
print("Notice how Cesium time changes with the simulation, while system time tracks real-world time.\n")

widget_clock

**Key observations:**
- **System time** (`timestamp`) changes based on when you interact (real-world time)
- **Cesium time** (`clock.current_time`) reflects the simulation time shown in the timeline
- When the animation plays, Cesium time advances according to the `multiplier` (speed)
- Click, move camera, or scrub the timeline to see both timestamps in action

In [ ]:
# Analyze clock events
if clock_events:
    from datetime import datetime
    
    print(f"\n{'='*70}")
    print(f"Collected {len(clock_events)} interaction events")
    print(f"{'=='*70}\n")
    
    # Show first few events
    for i, event in enumerate(clock_events[:5], 1):
        print(f"Event {i}: {event['type']}")
        print(f"  System: {event['system_time'][:19]}")
        print(f"  Cesium: {event['cesium_time'][:19] if event['cesium_time'] != 'N/A' else 'N/A'}")
        if event['clock_multiplier']:
            print(f"  Speed:  {event['clock_multiplier']}x")
        print()
    
    if len(clock_events) > 5:
        print(f"... and {len(clock_events) - 5} more events")
else:
    print("Interact with the map above to generate events!")

## 6. Advanced: Building a Flight Path

Create a flight path by clicking multiple points, then visualize it.

In [ ]:
widget6 = CesiumWidget(
    latitude=37.7749,
    longitude=-122.4194,
    altitude=5000,
    height="500px"
)

flight_path = []

def build_flight_path(event):
    if event['type'] == 'left_click' and 'picked_position' in event:
        pos = event['picked_position']
        flight_path.append([pos['longitude'], pos['latitude'], pos['altitude']])
        print(f"✈️  Waypoint {len(flight_path)}: ({pos['latitude']:.4f}, {pos['longitude']:.4f}, {pos['altitude']:.0f}m)")
        
        # Draw the path so far
        if len(flight_path) >= 2:
            geojson = {
                "type": "FeatureCollection",
                "features": [
                    {
                        "type": "Feature",
                        "geometry": {
                            "type": "LineString",
                            "coordinates": flight_path
                        },
                        "properties": {
                            "name": "Flight Path",
                            "stroke": "#FF0000",
                            "stroke-width": 3
                        }
                    }
                ]
            }
            widget6.load_geojson(geojson)

widget6.on_interaction(build_flight_path)

print("Click on the map to add waypoints to your flight path!")
widget6

In [ ]:
# Calculate total distance (simple approximation)
if len(flight_path) >= 2:
    import math
    
    def haversine_distance(coord1, coord2):
        """Calculate distance between two points in meters."""
        lon1, lat1 = coord1[0], coord1[1]
        lon2, lat2 = coord2[0], coord2[1]
        
        R = 6371000  # Earth radius in meters
        phi1, phi2 = math.radians(lat1), math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        
        a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        
        return R * c
    
    total_distance = sum(
        haversine_distance(flight_path[i], flight_path[i+1])
        for i in range(len(flight_path) - 1)
    )
    
    print(f"\n✈️  Flight Path Summary:")
    print(f"   Waypoints: {len(flight_path)}")
    print(f"   Total distance: {total_distance/1000:.2f} km")
else:
    print("Click at least 2 points to create a flight path!")

## 7. Export Interaction Data

Save interaction events to a file for later analysis.

In [ ]:
widget7 = CesiumWidget(
    latitude=48.8566,
    longitude=2.3522,
    altitude=5000,
    height="500px"
)

# Store all events with full details
all_events = []

def record_all_events(event):
    all_events.append(event)
    print(f"📝 Recorded {event['type']} event (total: {len(all_events)})")

widget7.on_interaction(record_all_events)

widget7

In [ ]:
# Export to JSON
if all_events:
    import json
    from pathlib import Path
    
    output_file = Path('interaction_log.json')
    with open(output_file, 'w') as f:
        json.dump(all_events, f, indent=2)
    
    print(f"✓ Exported {len(all_events)} events to {output_file}")
    print(f"  File size: {output_file.stat().st_size} bytes")
else:
    print("No events to export. Interact with the map first!")

## Summary

The `on_interaction()` callback provides powerful event handling for:

1. **Camera tracking** - Monitor user navigation and exploration
2. **Click handling** - Respond to user selections and inputs
3. **Timeline events** - Track animation playback and scrubbing
4. **Data collection** - Record user interactions for analysis
5. **Interactive applications** - Build responsive map-based tools

### Event Data Structure

Every event includes:
```python
{
    'type': 'camera_move' | 'left_click' | 'right_click' | 'timeline_scrub',
    'timestamp': '2024-01-01T12:00:00Z',  # System time (real-world)
    'camera': {
        'latitude': float,
        'longitude': float,
        'altitude': float,
        'heading': float,
        'pitch': float,
        'roll': float
    },
    'clock': {  # Cesium clock state (always included)
        'current_time': str,  # Simulation time from Cesium's clock
        'multiplier': float,  # Clock speed (e.g., 60 = 1 minute per second)
        'is_animating': bool  # Whether timeline is playing
    },
    'picked_position': {  # For click events only
        'latitude': float,
        'longitude': float,
        'altitude': float
    },
    'picked_entity': {...}  # When clicking on an entity
}
```

### Two Timestamps Available

- **`timestamp`**: System time when the interaction occurred (real-world time)
- **`clock.current_time`**: Cesium clock's simulation time (useful for time-dynamic data)

This dual timestamp system lets you correlate user interactions with both real-world time and simulation time in CZML animations.